In [3]:
import pandas as pd
import os
import pysam
import ast

In [9]:
sampleDictionary = {}
for file in os.listdir('/home/mark/Desktop/HLA/Globus/cleaned/'):
    if '.fasta.fai' in file:
        continue
    else:
        sample = file.split(".")[0]
        hap = file.split(".asm-")[1].split(".")[0]
        sampleDictionary[sample+"."+hap]='/home/mark/Desktop/HLA/Globus/cleaned/'+file

In [10]:
sampleDictionary

{'NA19836.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/NA19836.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00268.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG00268.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG03248.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/HG03248.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG02587.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/HG02587.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'NA19384.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/NA19384.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'NA19317.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/NA19317.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG02554.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG02554.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'NA19129.hap2': '/home/mark/Desktop/HLA/Globus/cleaned/NA19129.vrk-ps-sseq.asm-hap2.HLA.cleaned.fasta',
 'HG00514.hap1': '/home/mark/Desktop/HLA/Globus/cleaned/HG00514.vrk-ps-sseq.asm-hap1.HLA.cleaned.fasta',
 'HG00731.hap2': '/home/mark/Desktop/HLA/Globus/cleaned

In [7]:
immunoAnnotDict = {}
directory='/home/mark/Desktop/HLA/HengLi/Runs/'
for file in os.listdir(directory):
    immunoAnnotDict[str(file.split(".")[0])+".hap"+str(file.split(".")[1])]={}
    dfIA = pd.read_csv(directory+file, comment='#', sep='\t', header=None)
    dfIA2 = dfIA[dfIA[2]=='exon'].copy()
    for row in dfIA2.index:
        if 'DRA' in dfIA2.at[row,8]:
            geneName = ast.literal_eval(str(dfIA2.at[row,8].split("gene_name ")[1].split(";")[0]))
            start=str(dfIA2.at[row,3])
            end=str(dfIA2.at[row,4])
            immunoAnnotDict[str(file.split(".")[0])+".hap"+str(file.split(".")[1])][str(start)+"-"+str(end)]=geneName
        else:
            continue
immunoAnnotDict

{'HG02666.hap2': {'4390531-4390971': 'HLA-DRA',
  '4393385-4393630': 'HLA-DRA',
  '4394122-4394403': 'HLA-DRA',
  '4394693-4396241': 'HLA-DRA'},
 'HG02059.hap2': {'4325443-4325883': 'HLA-DRA',
  '4328298-4328543': 'HLA-DRA',
  '4329035-4329316': 'HLA-DRA',
  '4329606-4331146': 'HLA-DRA'},
 'NA18534.hap1': {'4311073-4311513': 'HLA-DRA',
  '4313927-4314172': 'HLA-DRA',
  '4314664-4314945': 'HLA-DRA',
  '4315235-4316783': 'HLA-DRA'},
 'NA19240.hap1': {'4369931-4370371': 'HLA-DRA',
  '4372785-4373030': 'HLA-DRA',
  '4373522-4373803': 'HLA-DRA',
  '4374093-4375633': 'HLA-DRA'},
 'NA19317.hap1': {'4264923-4265363': 'HLA-DRA',
  '4267777-4268022': 'HLA-DRA',
  '4268514-4268795': 'HLA-DRA',
  '4269085-4270633': 'HLA-DRA'},
 'HG02818.hap1': {'4403535-4403975': 'HLA-DRA',
  '4406388-4406633': 'HLA-DRA',
  '4407125-4407406': 'HLA-DRA',
  '4407696-4409202': 'HLA-DRA'},
 'HG03683.hap2': {'4276661-4277101': 'HLA-DRA',
  '4279515-4279760': 'HLA-DRA',
  '4280252-4280533': 'HLA-DRA',
  '4280823-4282371

In [14]:
immunoAnnotDict['chm13.hap1']

{'4292517-4292957': 'HLA-DRA',
 '4295371-4295616': 'HLA-DRA',
 '4296108-4296389': 'HLA-DRA',
 '4296679-4298227': 'HLA-DRA'}

In [15]:
myDRA_Sequences={}
for sample in immunoAnnotDict.keys():
    
    mygeneSeqs=[]
    for exon in immunoAnnotDict[sample]:
        
        start = int(exon.split("-")[0])
        end = int(exon.split("-")[1])
        
        sequence = ''.join(pysam.faidx(sampleDictionary[sample], sample+":"+str(start)+"-"+str(end)).split()[1:])
        mygeneSeqs.append(sequence)

    myDRA_Sequences[sample+":"+str(start)+"-"+str(end)]=''.join(mygeneSeqs)

In [16]:
myDRA_Sequences

{'HG02666.hap2:4394693-4396241': 'TCACTAATGTGCTTCAGGTATATCCCTGTCTAGAAGTCAGATTGGGGTTAAAGAGTCTGTCCGTGATTGACTAACAGTCTTAAATACTTGATTTGTTGTTGTTGTTGTCCTGTTTGTTTAAGAACTTTACTTCTTTATCCAATGAACGGAGTATCTTGTGTCCTGGACCCTTTGCAAGAACCCTTCCCCTAGCAACAGATGCGTCATCTCAAAATATTTTTCTGATTGGCCAAAGAGTAATTGATTTGCATTTTAATGGTCAGACTCTATTACACCCCACATTCTCTTTTCTTTTATTCTTGTCTGTTCTGCCTCACTCCCGAGCTCTACTGACTCCCAAAAGAGCGCCCAAGAAGAAAATGGCCATAAGTGGAGTCCCTGTGCTAGGATTTTTCATCATAGCTGTGCTGATGAGCGCTCAGGAATCATGGGCTATCAAAGAAGAACATGTGATCATCCAGGCCGAGTTCTATCTGAATCCTGACCAATCAGGCGAGTTTATGTTTGACTTTGATGGTGATGAGATTTTCCATGTGGATATGGCAAAGAAGGAGACGGTCTGGCGGCTTGAAGAATTTGGACGATTTGCCAGCTTTGAGGCTCAAGGTGCATTGGCCAACATAGCTGTGGACAAAGCCAACCTGGAAATCATGACAAAGCGCTCCAACTATACTCCGATCACCAATGTACCTCCAGAGGTAACTGTGCTCACGAACAGCCCTGTGGAACTGAGAGAGCCCAACGTCCTCATCTGTTTCATCGACAAGTTCACCCCACCAGTGGTCAATGTCACGTGGCTTCGAAATGGAAAACCTGTCACCACAGGAGTGTCAGAGACAGTCTTCCTGCCCAGGGAAGACCACCTTTTCCGCAAGTTCCACTATCTCCCCTTCCTGCCCTCAACTGAGGACGTTTACGACTGCAGGGTGGAGCACTGGGGCTTGGATGAGCCTCTTCTCAAGCACT

In [17]:
#with open('/home/mark/Desktop/HLA/DRA_Sequences.fasta', 'a+') as outFile:
#    for key in myDRA_Sequences.keys():
#        outFile.write(">"+str(key)+"\n")
#        outFile.write(str(myDRA_Sequences[key])+"\n")
#outFile.close()